In [1]:
import numpy as np
import warp as wp
import random
import math

In [2]:
class LognormalRandom:
    def __init__(self, seed=None):
        """
        Creates a lognormal RNG for whisker length generation

        Parameters:
        """
        self.rng = random
        # Mu and Sigma are defined in tens of microns.
        self.mu = None
        self.sigma = None

    def set_mu(self, mu: float) -> None:
        """
        Sets the mean value of the lognormal distribution.

        Parameters:
            mu (float): Average whisker length in tens of microns
        """
        print("mu")
        self.mu = mu

    def set_sigma(self, sigma: float) -> None:
        """
        Sets the standard deviation of the lognormal distribution.

        Parameters:
            sigma (float): Standard deviation of whisker length in tens of microns
        """
        print("sigma")
        self.sigma=sigma

    def set_rng_seed(self, seed: int) -> None:
        """
        Sets the seed for random generation.

        Parameters:
            seed: Default for truly pseudorandom metal whisker
        """
        self.rng = random.Random(seed) if seed is not None else random

    def generate_lognormal_random(self):
        """
        Generates lognormal random whisker lengths from parameters self was generated using.
        """
        u1 = self.rng.random()
        u2 = self.rng.random()
        std_norm_rand = math.sqrt(-2.0 * math.log(u1)) * math.sin(2.0 * math.pi * u2)
        return self.mu + self.sigma * std_norm_rand


In [3]:
class WhiskerGen:
    def __init__(self, batch_size: int, num_particles: int, spawn_positions: wp.array, whisker_lens: wp.array):
        """
        Instantiates whisker object prior to being dropped.

        Parameters:
            batch_size (int): Number of metal whiskers processed for this batch.
            whisker_length (float): Length of whisker in microns.
            total_particles (int): Number of particles representing one whisker in the simulation.
            spawn_positions (wp.array): Warp array containing the spawn position of each whisker.
        """
        self.batch_size = batch_size
        self.num_particles = num_particles
        self.whisker_length = whisker_lens
        self.total_particles = batch_size * num_particles
        

        # Allocate warp arrays on the GPU
        self.positions = wp.zeros(self.total_particles, dtype=wp.vec3)      # Particle positional data
        self.velocities= wp.zeros(self.total_particles, dtype=wp.vec3)    # Particle initial velocity

        # Initialize batch of whiskers on GPU
        wp.launch(GPUKernels.init_whisker_kernel, dim = self.total_particles,
                  inputs=[self.positions, self.velocities, whisker_lens, num_particles, spawn_positions])
        wp.synchronize()


    def get_positions(self):
        return self.positions.np() 
        

In [4]:
class BoxSpawner:
    def __init__(self, x1, x2, y1, y2, z1, z2):
        """
        Creates a spawner in a rectangular prism.

        Parameters:
            x1 (float): Negative x bound.
            x2 (float): Positive x bound.
            y1 (float): Negative y bound.
            y2 (float): Positive y bound.
            z1 (float): Negative z bound.
            z2 (float): Positive z bound.
        """
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
        self.z1 = z1
        self.z2 = z2

    def spawn_at(self):
        """
        Generate random spawn position within the spawn box.

        Returns:
            wp.vec3: Random coordinate denoting the origin of a metal whiskers spawn.
        """

        x = random.uniform(self.x1, self.x2)
        y = random.uniform(self.y1, self.y2)
        z = random.uniform(self.z1, self.z2)
        return wp.vec3(x, y, z)
        

In [5]:
# TODO: Implement ParameterValidator class
class ParameterValidator:
    def __init__(self):
        pass
        """
        Analyze closest conductive area
        """

In [6]:
# TODO: Implement RecordResults class
class RecordResults:
    def __init__(self):
        pass

In [7]:
class GPUKernels:
    @staticmethod
    @wp.kernel
    def drop_kernel(positions: wp.array(dtype=wp.vec3),
                    velocities: wp.array(dtype=wp.vec3),
                    dt: (float)):
        
        # Id of the GPU core operating on this falling object.
        tid = wp.tid()[0]
        # Subtracts the acceleration of gravity every second the kernel falls.
        gravity = wp.vec3(0.0, -9.81, 0.0)
        # Update the position of this whisker.
        positions[tid] = positions[tid] + velocities[tid] * dt

    @staticmethod
    @wp.kernel
    def init_whisker_kernel(positions: wp.array(dtype=wp.vec3),
                            velocities: wp.array(dtype=wp.vec3),
                            whisker_lens: wp.array(dtype=wp.float32),
                            num_particles: int,
                            spawn_positions: wp.array(dtype=wp.vec3)
                           ) -> None:
        
        tid = wp.tid()
        whisker_idx = tid // num_particles
        particle_idx = tid % num_particles

        whisker_len = whisker_lengths[whisker_idx]
        spacing = whisker_len / (num_particles - 1)
        
        spawn_offset = spawn_positions[whisker_idx]
        positions[tid] = wp.vec3(0.0, length - tid * spacing, 0.0)
        velocities[tid] = wp.vec3(0.0, 0.0, 0.0)

In [8]:
from typing import Optional, Tuple

# TODO: IMPLEMENT VALIDATOR LOGIC

class SimulationSettings:
    """
    Instantiates a class to manage the state of the simulation settings: batch size, box spawner, num whiskers, and PCB.
    """
    def __init__(self):
        self._batch_size: Optional[int] = None
        self._box_spawner = None
        self._num_whiskers: Optional[int] = None
        self._pcb = None

    # Maintain alphabetical order.
    
    @property
    def batch_size(self) -> Optional[int]:
        """Getter for batch size."""
        return self._batch_size

    @batch_size.setter
    def batch_size(self, batch_size: int) -> None:
        """
        Sets the size of whisker batches to run through the simulation at the same time.

        Parameters:
            batch_size: Number of whiskers to be simulated at once.
        """
        self._batch_size = batch_size

    @property
    def box_spawner(self):
        """Getter for spawn box"""
        return self._box_spawner

    @box_spawner.setter
    def box_spawner(self, box_spawner) -> None:
        """
        Sets the SpawnBox dimensions for this simulation

        Parameters:
            dims (tuple): A tuple with 6 floating points representing
            [x1, x2, y1, y2, z1, z2] for the spawn box bounds where
            {var}1 <= {var}2
        """
        # Model uses a model instead of the controller to create
        self.box_spawner = box_spawner

    @property
    def num_whiskers(self) -> Optional[int]:
        """Return number of whiskers per simulation."""
        return self._num_whiskers

    @num_whiskers.setter
    def num_whiskers(self, num_whiskers: int) -> None:
        """
        Sets the total number of whiskers to be dropped in this simulation.

        Parameters:
            num_whiskers (int): Sum of whiskers to be dropped.
        """
        self._num_whiskers = num_whiskers

    @property
    def pcb(self):
        """Get the data abstraction of the PCB object"""
        return self._pcb

    # TODO: Implement set_pcb method.
    @pcb.setter
    def pcb(self, path) -> None:
        """
        Parses a selected ICP file to create a PCB data abstraction in the simulation.
        set_pcb will then send static positional data to the view.

        Parameters:
            pcb_icp_file (object): ICP object to parse
        """
        self._pcb = None


In [9]:
from typing import Optional

# TODO: IMPLEMENT VALIDATOR LOGIC

class WhiskerSettings:
    def __init__(self):
        """Manages the state of the whisker settings: mu, sigma, and num particles."""
        self._mu = None
        self._sigma = None
        self._num_particles = None

    # Maintain alphabetical order.
            
    @property
    def mu(self) -> Optional[float]:
        """Getter for the average lognormal whisker length mu."""
        return self._mu
    
    @mu.setter        
    def mu(self, mu: float) -> None:
        """
        Setter for the average lognormal whisker length mu.

        Parameters:
            mu (float): Average length of whiskers.
        """
        self._mu = mu

    @property
    def num_particles(self) -> Optional[int]:
        """Getter for number of particles in one whisker."""
        return self._num_particles

    @num_particles.setter
    def num_particles(self, num_particles: int) -> None:
        """
        Setter for number of particles in one whisker.

        Param:
            num_particles (int): Number of particles in a metal whisker
        """
        self._num_particles = num_particles

    @property
    def sigma(self) -> Optional[float]:
        """Getter for the lognormal distribution standard deviation sigma"""
        return self._sigma

    @sigma.setter
    def sigma(self, sigma: float) -> None:
        """
        Setter for the lognormal distribution standard deviation sigma 
        
        Parameters:
            sigma: (float): Standard deviation of whiskers.
        """
        self._sigma = sigma


In [11]:
from typing import Optional, Tuple

class Controller:
    def __init__(self) -> None:
        """Orchestrates the usage of methods across classes."""
        # Instantiate validator to ensure graceful program use
        self.validator = ParameterValidator()
        self.simulation_settings = SimulationSettings()
        self.whisker_settings = WhiskerSettings()
        self.lrng = LognormalRandom()


# Set Simulation Settings ------------------------------------------------------ Alphabetical Ordering

    def set_batch_size(self, batch_size: int) -> None:
        """
        Sets the number of whiskers to instantiate and drop at one time.

        Parameters:
            batch_size (int): Number of whiskers to instantiate at once.
        """
        self.simulation_settings.batch_size = batch_size

    def set_box_spawner(self, dims: tuple) -> None:
        """
            Creates and sends a valid spawn box for the WhiskerSetting class to maintain.

            Parameters:
                dims (tuple): A tuple with 6 floating points representing
                [x1, x2, y1, y2, z1, z2] for the spawn box bounds where
                {var}1 <= {var}2
        """
        x1, x2, y1, y2, z1, z2 = dims
        box_spawner = BoxSpawner(x1, x2, y1, y2, z1, z2)
        self.simulation_settings.box_spawner = box_spawner

    def set_num_whiskers(self, num_whiskers: int):
        """
        Sets the number of whiskers to be instantiated on one simulation.

        Parameters (int):
            Number of whiskers to instantiate in one simulation.
        """

    def set_pcb(self, pcb_path):
        # TODO: Create parsing logic for IPC-2581 files.
        pass

# Set Whisker settings --------------------------------------------------------- Alphabetical Ordering

    def set_mu(self, mu: float) -> None:
        """Sets average whisker length in the WhiskerSettings class."""
        self.whisker_settings.mu = mu
        self.lrng.set_mu(mu)

    def set_num_particles(self, num_particles: int) -> None:
        """
        Sets the number of particles to represent a whisker for the next simulation.

        Parameters:
            num_particles (int): Number of particles to represent a whisker.
        """
        self.whisker_settings.num_particles = num_particles
        self.lrng.set_sigma(sigma)

    def set_sigma(self, sigma: float) -> None:
        """
        Sets the standard deviation in the WhiskerSettings class

        parameters:
            sigma: Standard deviation for the lognormal distribution.
        """
        self.whisker_settings.sigma = sigma
        self.lrng.set_sigma(sigma)

# Simulation Runner ----------------------------------------------------------------------------------------
    
    def run_simulation(self) -> None:
        """
        Starts a single physics simulation batch:
            - Instantiates batch of whiskers.
            - Spawn Whiskers at random locations in the spawn box.
            - Use lognormal RNG to determine whisker length.
            - Updates physics via GPU kernels.
        """
        # Get simulation settings:
        batch_size = self.simulation_settings.batch_size
        spawn_box = self.simulation_settings.spawn_box
        total_whiskers = self.simulation_settings.total_whiskers
        num_particles = self.whisker_settings.num_particles

        for current_batch_size in self._batch(total_whiskers, batch_size):
            # Generate lognormal random lengths for batch.
            whiskers_lens = np.array(
                [self.lrng.generate_lognormal_random() for _ in range(current_batch_size)], 
                dtype=np.float32
            )
            whiskers_lens = wp.from_numpy(whisker_lens, dtype=float32)

            # Generate spawn positions for each whisker in batch.
            spawn_positions_list = [spawn_box.spawn_at() for _ in range(current_batch_size)]
            spawn_positions = wp.array(spawn_positions_list, dtype=wp.vec3)

            # Instantiate batch of whiskers.
            whisker_batch = WhiskerGen(current_batch_size, num_particles, spawn_positions, whisker_lengths)

    # Helper batching method
    @staticmethod
    def _batch(total: int, batch: int):
        for start in range(0, total, batch):
            current_batch_size = min(batch, total - start)
            yield current_batch_size
 

      


In [ ]:
if __name__ == "__main__":
    controller = Controller()
    box = (-5, 5, 10, 20, -5, 5)
    print("loaded box")
    controller.set_box_spawner(box)
    print("loaded Controller")
    controller.set_batch_size(50)
    print("Set batch size")
    controller.set_num_whisker(200)
    print("Set total whiskers")
    controller.set_num_particles(7)
    print("set particles")
    controller.set_mu(4.5)
    print("set mu")
    controller.set_sigma(0.7)
    print("set sigm")